## Load data
The ca-GRQC data comes from [here](http://networkrepository.com/ca-GrQc.php#), using mmread funtion to load data.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from scipy.io import mmread
a = mmread('../data/ca-GrQc.mtx')

Note that, the .mtx file has a head line staring with %% or %. **a** is an object of scipy.sparse.coo_matrix.

We can directly convert scipy.sparse.coo_matrix to Graph object for networkx using nx.Graph(). G.number_of_nodes() shows the number of vetices, i.e., **order**. G.number_of_edges() shows the number of edges, i.e., **size**.

In [ ]:
import networkx as nx
G=nx.Graph(a)
G.number_of_edges()
G.number_of_nodes()

The default node starts from 0, one can use relabel_nodes() to change the node labels. Below is the usage of relabel_nodes(), note that the mapping function.

In [ ]:
mapping = dict(zip(G, range(1, G.number_of_nodes()+1)))
G = nx.relabel_nodes(G, mapping)
sorted(G)[:3]

Clearly, the node's label starts from 1.

In [ ]:
nx.is_weighted(G)
nx.is_directed(G)

## Show the Graph

The graph **G** is too huge to display. Here, we first extract a subgraph, then show it.

In [ ]:
node_degree_max = max(dict(G.degree()).items(), key=lambda x: x[1])
node_edges = G.edges(node_degree_max[0])


In [ ]:
import numpy as np 
a = [n for n in G.neighbors(node_degree_max[0])]
a.append(node_degree_max[0])

H = G.subgraph(a)
import matplotlib.pyplot as plt 
nx.draw(H)


In [ ]:
# or using the subgraph method
node_sub = list(G.nodes())[:100]
a = nx.adjacency_matrix(H1)
print(a)
H1 = nx.subgraph(G, node_sub)
nx.draw(H1)
plt.show()

There is another example to find the node with largest degree, see [here](https://networkx.org/documentation/stable//auto_examples/drawing/plot_ego_graph.html#sphx-glr-auto-examples-drawing-plot-ego-graph-py)

In [ ]:
from operator import itemgetter
node_and_degree = G.degree()
(largest_hub, degree) = sorted(node_and_degree, key=itemgetter(1))[-1]
# Create ego graph of main hub
hub_ego = nx.ego_graph(G, largest_hub)
# Draw graph
pos = nx.spring_layout(hub_ego)
nx.draw(hub_ego, pos, node_color="b", node_size=50, with_labels=False)
# Draw ego as large and red
options = {"node_size": 300, "node_color": "r"}
nx.draw_networkx_nodes(hub_ego, pos, nodelist=[largest_hub], **options)
plt.show()